<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/portfolio/nlp/fine-tuned-llm/psy_ai_wandb_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Psy AI
Psy AI is a Llama3-8b model instruction fine-tuned on depression dataset and is meant to help improve mental well-being.

In future iterations it is meant to be trained on multiple philosophical and psychological datasets in order to provide multifaceted answers to complex mental health issues.



# Datasets

Philosophy datasets (* for future training)
* https://www.kaggle.com/datasets/christopherlemke/philosophical-texts
* https://www.workwithdata.com/object/philosophy-science-complete-a-text-on-traditional-problems-schools-thought-book-by-edwin-h-c-hung-0000
* https://www.kaggle.com/datasets/christopherlemke/philosophy-authors-writings-german
* https://www.workwithdata.com/object/philosophical-inquiries-an-introduction-to-problems-philosophy-book-by-nicholas-rescher-0000
* https://www.workwithdata.com/object/roman-stoicism-book-by-edward-vernon-arnold-1857
* https://www.workwithdata.com/object/wisdom-energy-basic-buddhist-teachings-book-by-thubten-yeshe-1935

Psychology and mental health datasets

* Kaggle Depression data for chatbot https://www.kaggle.com/datasets/nupurgopali/depression-data-for-chatbot
* Kaggle Psychometrics dataset https://www.kaggle.com/discussions/general/304994
* Psychometric tests dataset https://ieee-dataport.org/documents/psychometric-tests-dataset
* Psychometric NLP https://paperswithcode.com/dataset/psychometric-nlp
* Reddit mental health dataset https://zenodo.org/record/3941387
* Reddit mental disorders identification https://www.kaggle.com/datasets/kamaruladha/mental-disorders-identification-reddit-nlp
* Kaggle Mental Health Conversational Data https://www.kaggle.com/datasets/elvis23/mental-health-conversational-data
* Kaggle Mental Health FAQ for Chatbot https://www.kaggle.com/narendrageek/mental-health-faq-for-chatbot/code
* A human consciousness questionnaire dataset https://data.mendeley.com/datasets/69p62ksdh6
* paperswithcode Self-reported Mental Health Diagnoses https://paperswithcode.com/dataset/smhd
* paperswithcode Mental Health Summarization Dataset https://paperswithcode.com/dataset/mentsum
* HuggingFace psychology dataset https://huggingface.co/datasets/samhog/psychology-10k


In [1]:
!pip install -U -q gradio
!pip install -U -q datasets
!pip install -U -q bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -U -q trl

!pip install -U -q evaluate
!pip install -U -q rouge_score
!pip install -U -q optuna
!pip install -U -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.8 MB/s eta

In [2]:
import os
from datasets import load_dataset
import json
import yaml
import gradio as gr
import torch
import transformers
from transformers import GenerationConfig, Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModel, AutoPeftModelForCausalLM
import numpy as np
from evaluate import load
import optuna
import wandb
import datetime

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import wandb
wandb.login()
!wandb launch-agent -e vitaliy-sharandin-org -q "Starter queue"

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Starting launch agent ✨
wandb: WARNING launch: Agent is running on team entity (vitaliy-sharandin-org). Members of this team will be able to run code on this device.
wandb: launch: agent 34cmvypc polling on queues Starter queue, running 0 out of a maximum of 1 jobs
wandb: launch: Launch agent received job:
wandb: {'runQueueItemId': 'UnVuUXVldWVJdGVtOjU1OTk4NTIzMw==',
wandb:  'runSpec': {'_wandb_job_collection_id': 'QXJ0aWZhY3RDb2xsZWN0aW9uOjE3MTQxNzU3NA==',
wandb:              'author': 'vitaliy-sharandin',
wandb:              'entity': 'vitaliy-sharandin-org',
wandb:              'job': 'vitaliy-sharandin-org/Psy-AI/psy-ai:latest',
wandb:              'overrides': {'args': [],
wandb:                            'entry_point': [],
wandb:                            'run_config': {'_name_or_path': 'NousResearch/Meta-Llama-3-8B-Instruct',
wandb:                                           'accelerator_config': {'dispatch_batches': None,
wandb:                                          

In [5]:
import wandb
wandb.login()

job = """
import os
from datasets import load_dataset
import json
import yaml
import gradio as gr
import torch
import transformers
from transformers import GenerationConfig, Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModel, AutoPeftModelForCausalLM
import numpy as np
from evaluate import load
import optuna
import wandb
import datetime

import warnings
warnings.filterwarnings('ignore')

depression_dataset = load_dataset("vitaliy-sharandin/depression-instruct")

def formatting_func_train(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n"
      f"{example['response']}")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response:\n"
      f"{example['response']}")

  return {"text" : input_prompt}

def formatting_func_test(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response: \n")

  return {"text" : input_prompt}

formatted_depression_dataset_train = depression_dataset['train'].map(formatting_func_train).select(range(3))
formatted_depression_dataset_test = depression_dataset['train'].map(formatting_func_test).select(range(3))

model_name = "NousResearch/Meta-Llama-3-8B-Instruct"

def get_tokenizer():
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = "right"
  return tokenizer

def get_model():


  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  qlora_config = LoraConfig(lora_alpha=16,
                          lora_dropout=0.1,
                          r=64,
                          bias="none",
                          task_type="CAUSAL_LM")

  base_training_model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config=bnb_config,
      device_map = {"": 0}
  )

  base_training_model = prepare_model_for_kbit_training(base_training_model)
  base_training_model = get_peft_model(base_training_model, qlora_config)
  return base_training_model.to('cuda')

base_training_model = get_model()
tokenizer = get_tokenizer()

torch.manual_seed(42)
print(base_training_model)

entity="vitaliy-sharandin-org"
project_name="Psy-AI"
job_name = "psy-ai"

config={"epoch":1}
settings = wandb.Settings(job_name=job_name)
os.environ["WANDB_LOG_MODEL"]="true"

model_trained_checkpoint = 'llama3-trained'

def bleu_rouge_f1_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=-1)

  labels = [[idx for idx in label if idx != -100] for label in labels]
  predictions = [[idx for idx in prediction if idx != -100] for prediction in predictions]

  decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  bleu = load("bleu")
  bleu_results = bleu.compute(predictions=decoded_predictions, references=decoded_labels)

  rouge = load('rouge')
  rouge_results = rouge.compute(predictions=decoded_predictions, references=decoded_labels)

  f1 = 2 * (bleu_results['bleu'] * rouge_results['rouge1']) / (bleu_results['bleu'] + rouge_results['rouge1'])

  scores = {
        "bleu": bleu_results["bleu"],
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "f1": f1
    }

  wandb.log(scores)

  return scores

def fine_tune(model, tokenizer, train_dataset, eval_dataset, metrics_func, only_evaluate=False):

  supervised_finetuning_trainer = SFTTrainer(model=model,
                                            train_dataset=train_dataset,
                                            eval_dataset=eval_dataset,
                                            args=TrainingArguments(
                                                output_dir="./training_results",
                                                report_to="wandb",
                                                per_device_train_batch_size=8,
                                                per_device_eval_batch_size=1,
                                                gradient_accumulation_steps=2,
                                                ddp_find_unused_parameters=False,
                                                optim="paged_adamw_8bit",
                                                save_steps=1000,
                                                logging_steps=30,
                                                learning_rate=2e-4,
                                                weight_decay=0.001,
                                                fp16=False,
                                                bf16=False,
                                                max_grad_norm=0.3,
                                                max_steps=-1,
                                                warmup_ratio=0.3,
                                                group_by_length=True,
                                                lr_scheduler_type="constant"
                                            ),
                                            dataset_text_field="text",
                                            max_seq_length=2048,
                                            compute_metrics=metrics_func,
                                            data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False))

  if only_evaluate:
    return supervised_finetuning_trainer.evaluate()

  else:
    with wandb.init(
      entity=entity, config=config, project=project_name, settings=settings
    ) as run:
      supervised_finetuning_trainer.train()
      eval_results = supervised_finetuning_trainer.evaluate()

      run.log_code()
      wandb.finish()

      return eval_results

params = {
    "model": base_training_model,
    "tokenizer": tokenizer,
    "train_dataset": formatted_depression_dataset_train,
    "eval_dataset": formatted_depression_dataset_train,
    "metrics_func": bleu_rouge_f1_metrics
}

fine_tune(**params)
"""

with open('job.py', 'w+') as file:
  file.write(job)

requirements = """
gradio
datasets
bitsandbytes
transformers
peft
accelerate
trl
evaluate
rouge_score
optuna
wandb
"""

with open('requirements.txt', 'w+') as file:
  file.write(requirements)



<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


  File "/content/job.py", line 27
    "Write a response that appropriately completes the request.
    ^
SyntaxError: unterminated string literal (detected at line 27)


In [6]:
!python job.py

2024-05-06 21:54:28.566213: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 21:54:28.566259: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 21:54:28.567520: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 21:54:29.799109: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Generating train split: 100% 51/51 [00:00<00:00, 1871.92 examples/s]
Map: 100% 51/51 [00:00<00:00, 4748.69 examples/s]
Map: 100% 51/51 [00:00<00:00, 10784.99 examples/s]
config.json: 100% 654/654 [00:00<00:00, 4.80MB/s]
model.safetensors.ind

# Dataset instruction transformation

Depression dataset with 51 q&a entries was taken for mvp to see if model is indeed training.

In [3]:
depression_dataset = load_dataset("vitaliy-sharandin/depression-instruct")

Generating train split:   0%|          | 0/51 [00:00<?, ? examples/s]

First, we modify our dataset to Alpaca format and create two datasets. One - for testing and evaluation and second one for inference, where responses are not available in formatted prompt.

In [4]:
def formatting_func_train(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n"
      f"{example['response']}")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response:\n"
      f"{example['response']}")

  return {"text" : input_prompt}

def formatting_func_test(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response: \n")

  return {"text" : input_prompt}

In [5]:
formatted_depression_dataset_train = depression_dataset['train'].map(formatting_func_train).select(range(3))
formatted_depression_dataset_test = depression_dataset['train'].map(formatting_func_test).select(range(3))

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

# Model load

We are loading our Llama2 model in 4bit quantized form as well as applying Lora for peft training.

In [6]:
model_name = "NousResearch/Meta-Llama-3-8B-Instruct"

def get_tokenizer():
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = "right"
  return tokenizer

def get_model():


  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  qlora_config = LoraConfig(lora_alpha=16,
                          lora_dropout=0.1,
                          r=64,
                          bias="none",
                          task_type="CAUSAL_LM")

  base_training_model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config=bnb_config,
      device_map = {"": 0}
  )

  base_training_model = prepare_model_for_kbit_training(base_training_model)
  base_training_model = get_peft_model(base_training_model, qlora_config)
  return base_training_model.to('cuda')

base_training_model = get_model()
tokenizer = get_tokenizer()

torch.manual_seed(42)
print(base_training_model)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=F

# Model instruction fine-tuning

Here we are defining inference function which returns bleu, rouge and f1 metrics after comparison of predicted and reference responses. My tests have shown that standard trainer `compute_metrics` method is quite inefficient and is not quite suitable for instruction fine-tuning during manual observations or generated results.

In [13]:
entity="vitaliy-sharandin-org"
project_name="Psy-AI"
job_name = "psy-ai"

config={"epoch":1}
settings = wandb.Settings(job_name=job_name)
os.environ["WANDB_LOG_MODEL"]="true"

model_trained_checkpoint = 'llama3-trained'

def bleu_rouge_f1_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=-1)

  labels = [[idx for idx in label if idx != -100] for label in labels]
  predictions = [[idx for idx in prediction if idx != -100] for prediction in predictions]

  decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  bleu = load("bleu")
  bleu_results = bleu.compute(predictions=decoded_predictions, references=decoded_labels)

  rouge = load('rouge')
  rouge_results = rouge.compute(predictions=decoded_predictions, references=decoded_labels)

  f1 = 2 * (bleu_results['bleu'] * rouge_results['rouge1']) / (bleu_results['bleu'] + rouge_results['rouge1'])

  scores = {
        "bleu": bleu_results["bleu"],
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "f1": f1
    }

  wandb.log(scores)

  return scores

def fine_tune(model, tokenizer, train_dataset, eval_dataset, metrics_func, only_evaluate=False):

  supervised_finetuning_trainer = SFTTrainer(model=model,
                                            train_dataset=train_dataset,
                                            eval_dataset=eval_dataset,
                                            args=TrainingArguments(
                                                output_dir="./training_results",
                                                report_to="wandb",
                                                per_device_train_batch_size=8,
                                                per_device_eval_batch_size=1,
                                                gradient_accumulation_steps=2,
                                                ddp_find_unused_parameters=False,
                                                optim="paged_adamw_8bit",
                                                save_steps=1000,
                                                logging_steps=30,
                                                learning_rate=2e-4,
                                                weight_decay=0.001,
                                                fp16=False,
                                                bf16=False,
                                                max_grad_norm=0.3,
                                                max_steps=-1,
                                                warmup_ratio=0.3,
                                                group_by_length=True,
                                                lr_scheduler_type="constant"
                                            ),
                                            dataset_text_field="text",
                                            max_seq_length=2048,
                                            compute_metrics=metrics_func,
                                            data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False))

  if only_evaluate:
    return supervised_finetuning_trainer.evaluate()

  else:
    with wandb.init(
      entity=entity, config=config, project=project_name, settings=settings
    ) as run:
      supervised_finetuning_trainer.train()
      # supervised_finetuning_trainer.model.save_pretrained(model_trained_checkpoint)
      # tokenizer.save_pretrained(model_trained_checkpoint)
      eval_results = supervised_finetuning_trainer.evaluate()

      wandb.finish()

      return eval_results

## Evaluating base model

In [10]:
# params = {
#     "model": base_training_model,
#     "tokenizer": tokenizer,
#     "train_dataset": formatted_depression_dataset_train,
#     "eval_dataset": formatted_depression_dataset_train,
#     "metrics_func": bleu_rouge_f1_metrics,
#     "only_evaluate": True
# }

# fine_tune(**params)

Evaluation metrics are quite low before training.

Let's train the model and see how metrics and responses change.

## Fine-tuning base and evaluating tuned model

In [14]:
params = {
    "model": base_training_model,
    "tokenizer": tokenizer,
    "train_dataset": formatted_depression_dataset_train,
    "eval_dataset": formatted_depression_dataset_train,
    "metrics_func": bleu_rouge_f1_metrics
}

fine_tune(**params)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss


wandb: WARNING No relevant files were detected in the specified directory. No code will be logged to your run.


bleu,▁
eval/bleu,▁
eval/f1,▁
eval/loss,▁
eval/rouge1,▁
eval/rouge2,▁
eval/rougeL,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
f1,▁


{'eval_loss': 2.4444386959075928,
 'eval_bleu': 0.24486861515017747,
 'eval_rouge1': 0.6201960181988517,
 'eval_rouge2': 0.2799906622553873,
 'eval_rougeL': 0.5136170446886649,
 'eval_f1': 0.35111027371461906,
 'eval_runtime': 6.158,
 'eval_samples_per_second': 0.487,
 'eval_steps_per_second': 0.487,
 'epoch': 3.0}

We can see that all scores are significantly improved after training.

# Saving fine-tuned model with adapters to Huggingface

Now we are able to merge our model with saved peft adapters and push it to Hugging Face repo.

We have to launch this cell separately, as VRAM will be filled up to fullest point when we merge adapters with original model.

In [ ]:
# tuned_model = AutoPeftModelForCausalLM.from_pretrained(
#     model_trained_checkpoint,
#     low_cpu_mem_usage=True,
#     torch_dtype=torch.float16,
#     device_map = {"": 0}
# )
# tokenizer = AutoTokenizer.from_pretrained(model_trained_checkpoint)

# merged_model = tuned_model.merge_and_unload()

# merged_model.save_pretrained(model_merged, safe_serialization=True)
# tokenizer.save_pretrained(model_merged)

# mlflow.register_model(model_uri=model_merged, name="psy-ai")

# # token=''
# # merged_model.push_to_hub("vitaliy-sharandin/wiseai", token=token)
# # tokenizer.push_to_hub("vitaliy-sharandin/wiseai", token=token)

# Gradio demo in Hugging Face Spaces

https://huggingface.co/spaces/vitaliy-sharandin/WiseAI

## Local launch
Requirements:
* Choose free T4 GPU in Colab environment settings
* Run `pip install` and `imports` cells at the beginning before launching Gradio.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
      'vitaliy-sharandin/wiseai',
      load_in_8bit=True,
      device_map = {"": 0}
  )
tokenizer = AutoTokenizer.from_pretrained('vitaliy-sharandin/wiseai')

pipe = pipeline('text-generation', model=model,tokenizer=tokenizer)

def generate_text(instruction, input):
  if not instruction.strip():
    return str('The instruction field is required.')

  if instruction.strip() and input.strip():
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{instruction}\n\n"
      "### Input:\n"
      f"{input}\n\n"
      f"### Response: \n")
  else :
    input_prompt = (f"Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n"
        f"{instruction}\n\n"
        f"### Response: \n")
  result = pipe(input_prompt, max_length=200, top_p=0.9, temperature=0.9, num_return_sequences=1, return_full_text=False)[0]['generated_text']
  return result[:str(result).find("###")]

iface = gr.Interface(fn=generate_text, inputs=[gr.Textbox(label="Instruction"),
                                               gr.Textbox(label="Additional Input")],
                     outputs=gr.Textbox(label="Response"))
iface.launch()